<a href="https://colab.research.google.com/github/avynash/DesignOptimization2021Fall/blob/main/Homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).





Problem 1:Solution

In [8]:
import numpy as np
import torch as t
from torch.autograd import Variable

x1_info = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
p_info = [28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5]
p_wsat = 10.0**(8.07131-1730.63/(20.0+233.426))
p_dsat = 10.0**(7.43155-1554.679/(20.0+240.337))

def loss(a):
    total_loss = 0.0
    for i in range(11):
        x1 = x1_info[i]
        p = p_info[i]
        p_norm = x1*p_wsat*t.exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2) + (1-x1)*p_dsat*t.exp(a[1]*(a[0]*x1/(a[0]*x1+a[1]*(1-x1)))**2)
        total_loss = total_loss + (p_norm-p)**2
    return total_loss

error = 1
A = Variable(t.tensor([1.0,1.0]), requires_grad = True)
while error>= 0.05:
    loss(A).backward()
    error = t.linalg.norm(A.grad)
    s = .2
    while loss(A-s*A.grad) > loss(A):
        s = .5*s
    with t.no_grad():
        A -= s*A.grad
        A.grad.zero_()
print(A)
print(loss(A))


tensor([1.9582, 1.6893], requires_grad=True)
tensor(0.6702, grad_fn=<AddBackward0>)


In [15]:
from math import exp
a = [1.9582,1.6893]
for i in range(11):
    x1 = x1_info[i]
    p_norm = x1*p_wsat*exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2) + (1-x1)*p_dsat*exp(a[1]*(a[0]*x1/(a[0]*x1+a[1]*(1-x1)))**2)
    print((p_norm,3), p_info[i],((p_norm-p_info[i]/p_info[i],4)))

(28.824099527405245, 3) 28.1 (27.824099527405245, 4)
(34.64328584864464, 3) 34.4 (33.64328584864464, 4)
(36.452102883335144, 3) 36.7 (35.452102883335144, 4)
(36.86661716636917, 3) 36.9 (35.86661716636917, 4)
(36.87334015836798, 3) 36.8 (35.87334015836798, 4)
(36.74916213399144, 3) 36.7 (35.74916213399144, 4)
(36.38987768165168, 3) 36.5 (35.38987768165168, 4)
(35.38456535962058, 3) 35.4 (34.38456535962058, 4)
(32.94803191258182, 3) 32.9 (31.94803191258182, 4)
(27.730647340243344, 3) 27.7 (26.730647340243344, 4)
(17.47325208459706, 3) 17.5 (16.47325208459706, 4)


Problem 2: Solution

In [2]:
from bayes_opt import BayesianOptimization
from matplotlib import pyplot

def function(x, y):
    return -((4-2.1*x*2+(x4)/3)*x2+x*y+(-4+4*y2)*y*2)
pbounds = {'x': (-3, 3), 'y': (-2, 2)}

optimizer = BayesianOptimization(f=function,pbounds=pbounds,random_state=1)

optimizer.maximize(init_points=2,n_iter=15)
print(optimizer.max)

ModuleNotFoundError: ignored